In [ ]:
!pip install scikit-learn pandas numpy gradio nltk textblob

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import gradio as gr
import re
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

import nltk
try:
    nltk.download('punkt', quiet=True)
    nltk.download('stopwords', quiet=True)
    nltk.download('wordnet', quiet=True)
except:
    pass

from textblob import TextBlob

print("Loading offline chatbot (no model downloads required)...")

class OfflineInternshipChatbot:
    def __init__(self):
        print("Initializing chatbot...")

        self.vectorizer = TfidfVectorizer(
            max_features=1000,
            stop_words='english',
            ngram_range=(1, 2),
            lowercase=True
        )

        self.setup_knowledge_base()
        self.setup_offline_models()

        print("Chatbot initialized successfully!")

    def setup_knowledge_base(self):
        self.faq_data = [
            {
                "category": "Application Process",
                "question": "How do I apply for an internship?",
                "answer": "To apply for an internship: 1) Visit our careers page and create an account, 2) Upload your resume (PDF format) and write a compelling cover letter, 3) Fill out the complete application form with all required information, 4) Submit your application before the deadline, 5) Wait for confirmation email. Make sure your resume highlights relevant skills, projects, and experiences that match the internship role.",
                "keywords": ["apply", "application", "how to apply", "submit", "process", "steps"]
            },
            {
                "category": "Application Process",
                "question": "What documents do I need for internship application?",
                "answer": "Required documents include: Resume/CV (PDF format, max 2 pages), Cover letter (tailored to the specific role), Official academic transcripts, 2-3 letters of recommendation from professors or employers, Portfolio or work samples (if applicable to your field), Any relevant certifications or awards. Ensure all documents are clearly labeled and in PDF format.",
                "keywords": ["documents", "requirements", "resume", "cv", "transcripts", "recommendation letters", "portfolio"]
            },
            {
                "category": "Requirements",
                "question": "What are the eligibility requirements for internships?",
                "answer": "General eligibility requirements: Currently enrolled in an accredited degree program (Bachelor's or Master's), Minimum cumulative GPA of 3.0 on a 4.0 scale, Completed relevant coursework in your field of study, Available for full-time commitment during the internship period, Strong written and verbal communication skills, Demonstrated passion for learning and professional growth, Legal authorization to work in the country.",
                "keywords": ["eligibility", "requirements", "qualifications", "gpa", "enrolled", "student", "criteria"]
            },
            {
                "category": "Duration & Schedule",
                "question": "How long do internships last and what is the schedule?",
                "answer": "Internship durations vary by program: Summer internships typically last 10-12 weeks (May-August), Semester internships run 14-16 weeks (Fall/Spring), Part-time internships during academic year are flexible (20 hours/week), Full-time internships during breaks are 40 hours/week. Work hours are typically Monday-Friday, 9 AM - 5 PM, with some flexibility for different roles.",
                "keywords": ["duration", "length", "schedule", "time", "hours", "weeks", "summer", "semester"]
            },
            {
                "category": "Compensation",
                "question": "Are internships paid and what benefits do you offer?",
                "answer": "Yes, all our internships are paid positions! Compensation varies based on role, location, experience level, and academic year. In addition to competitive hourly wages, we provide: Professional mentorship programs, Networking opportunities with industry leaders, Access to training and development resources, Potential for full-time job offers, Transportation stipends (where applicable), Flexible work arrangements.",
                "keywords": ["paid", "salary", "stipend", "compensation", "benefits", "money", "wage", "payment"]
            },
            {
                "category": "Remote Work",
                "question": "Can I work remotely as an intern?",
                "answer": "We offer flexible work arrangements including remote, hybrid, and on-site options depending on the specific role and team needs. Remote interns receive the same level of support, mentorship, and resources as on-site interns. We provide necessary technology and equipment. Some roles may require occasional in-person meetings or training sessions. Technical requirements and home office setup guidelines will be provided upon acceptance.",
                "keywords": ["remote", "virtual", "online", "work from home", "hybrid", "flexible", "location"]
            },
            {
                "category": "Training",
                "question": "What training and development will I receive?",
                "answer": "Comprehensive training program includes: Week-long orientation covering company culture and systems, Technical skill workshops specific to your role, Professional development sessions on communication and leadership, Assigned mentorship with experienced professionals, Regular one-on-one feedback sessions with supervisors, Access to online learning platforms and courses, Industry conference attendance opportunities, Lunch-and-learn sessions with executives.",
                "keywords": ["training", "development", "learning", "skills", "workshop", "mentorship", "orientation"]
            },
            {
                "category": "Projects",
                "question": "What kind of projects will I work on as an intern?",
                "answer": "Interns work on meaningful, real-world projects including: Direct client work and customer solutions, Product development and feature implementation, Market research and competitive analysis, Data analysis and business intelligence projects, Marketing campaigns and content creation, Process improvement initiatives, Software development and testing, Research projects with potential for publication. All projects are designed to provide hands-on learning and measurable impact.",
                "keywords": ["projects", "work", "tasks", "assignments", "responsibilities", "experience", "real work"]
            },
            {
                "category": "Support",
                "question": "What support is available for interns?",
                "answer": "Comprehensive support system includes: Dedicated mentor assigned from day one, Regular check-ins with direct supervisors and HR team, Peer networking groups and intern cohort activities, 24/7 access to employee assistance programs, Professional development resources and career guidance, Mental health and wellness programs, Open-door policy with management, Technical support and IT assistance, Social events and team-building activities.",
                "keywords": ["support", "help", "assistance", "mentor", "guidance", "resources", "help desk"]
            },
            {
                "category": "Evaluation",
                "question": "How are interns evaluated and what feedback will I receive?",
                "answer": "Evaluation process includes: Mid-term performance review (week 6), Final comprehensive evaluation at program end, Project presentations to leadership team, 360-degree feedback from peers and supervisors, Self-assessment and goal reflection exercises, Skills development tracking and progress reports. All feedback is constructive, specific, and designed to support your professional growth and future career success.",
                "keywords": ["evaluation", "feedback", "review", "assessment", "performance", "grading", "progress"]
            },
            {
                "category": "Full-time Opportunities",
                "question": "Can internships lead to full-time job offers?",
                "answer": "Absolutely! Many of our interns receive full-time offers. Success factors include: Outstanding performance during internship period, Strong cultural fit with team and company values, Availability of full-time positions in your area, Alignment of graduation timeline with business needs, Continued mutual interest from both intern and company. We have a 70% conversion rate from internship to full-time employment. Even if immediate positions aren't available, we maintain relationships for future opportunities.",
                "keywords": ["full-time", "job offer", "permanent", "hire", "conversion", "employment", "career"]
            },
            {
                "category": "Timeline",
                "question": "When should I apply and what are the deadlines?",
                "answer": "Application timeline varies by program: Summer internships - Apply September through November of previous year, Fall internships - Apply February through April, Spring internships - Apply August through October, Rolling admissions available for some specialized programs. Early applications are strongly encouraged as positions fill quickly. We recommend applying at least 6-8 months in advance for summer programs.",
                "keywords": ["timeline", "deadline", "when to apply", "application period", "dates", "schedule"]
            },
            {
                "category": "Technical Requirements",
                "question": "What technical skills or software do I need to know?",
                "answer": "Technical requirements vary by role: General requirements include proficiency in Microsoft Office Suite, basic computer literacy, and email communication. Role-specific requirements: Engineering interns may need programming languages (Python, Java, C++), Design interns should know Adobe Creative Suite, Marketing interns benefit from social media platforms and analytics tools, Data science interns need SQL and statistical software. Don't worry if you don't know everything - we provide training!",
                "keywords": ["technical", "skills", "software", "programming", "computer", "technology", "tools"]
            },
            {
                "category": "Location & Housing",
                "question": "Where are internships located and is housing provided?",
                "answer": "Internship locations include our main headquarters, regional offices in major cities, and remote positions. For on-site internships, we provide: Housing stipends for out-of-state interns, Assistance finding local accommodations, Transportation benefits and parking, Information about local amenities and attractions, Relocation support for international students. Housing arrangements vary by location and are discussed during the offer process.",
                "keywords": ["location", "housing", "accommodation", "relocation", "office", "where", "city"]
            }
        ]

        self.knowledge_df = pd.DataFrame(self.faq_data)

        self.questions = [item['question'] for item in self.faq_data]
        self.answers = [item['answer'] for item in self.faq_data]
        self.all_keywords = []
        for item in self.faq_data:
            self.all_keywords.extend(item['keywords'])

    def setup_offline_models(self):
        print("Setting up text similarity models...")

        question_texts = [q.lower() for q in self.questions]
        self.question_tfidf_matrix = self.vectorizer.fit_transform(question_texts)

        print(f"Processed {len(self.questions)} questions for similarity matching")

    def preprocess_text(self, text):
        text = text.lower()
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
        text = ' '.join(text.split())
        return text

    def find_best_match(self, user_query, threshold=0.1):
        processed_query = self.preprocess_text(user_query)

        query_tfidf = self.vectorizer.transform([processed_query])

        similarities = cosine_similarity(query_tfidf, self.question_tfidf_matrix)[0]

        best_match_idx = np.argmax(similarities)
        best_similarity = similarities[best_match_idx]

        if best_similarity > threshold:
            return best_match_idx, best_similarity

        return None, 0

    def keyword_match(self, user_query):
        query_words = set(self.preprocess_text(user_query).split())

        best_matches = []
        for i, item in enumerate(self.faq_data):
            item_keywords = set([kw.lower() for kw in item['keywords']])
            overlap = len(query_words.intersection(item_keywords))
            if overlap > 0:
                best_matches.append((i, overlap))

        if best_matches:
            best_matches.sort(key=lambda x: x[1], reverse=True)
            return best_matches[0][0], best_matches[0][1]

        return None, 0

    def extract_intent(self, user_query):
        query_lower = user_query.lower()

        intents = {
            'greeting': ['hello', 'hi', 'hey', 'good morning', 'good afternoon', 'good evening'],
            'thanks': ['thank', 'thanks', 'appreciate', 'grateful'],
            'application': ['apply', 'application', 'submit', 'how to apply'],
            'requirements': ['requirement', 'eligible', 'qualification', 'criteria'],
            'timeline': ['when', 'deadline', 'timeline', 'apply by'],
            'compensation': ['pay', 'salary', 'paid', 'money', 'stipend'],
            'duration': ['long', 'duration', 'time', 'weeks', 'months'],
            'remote': ['remote', 'virtual', 'home', 'online'],
            'projects': ['project', 'work', 'do', 'tasks'],
            'support': ['help', 'support', 'mentor', 'assistance']
        }

        for intent, keywords in intents.items():
            if any(keyword in query_lower for keyword in keywords):
                return intent

        return 'unknown'

    def generate_response(self, user_query):
        if not user_query.strip():
            return "Hello! I'm here to help with your internship questions. What would you like to know?"

        intent = self.extract_intent(user_query)

        if intent == 'greeting':
            return "Hello! I'm your internship support assistant. I can help you with questions about applications, requirements, training, projects, compensation, and much more. What would you like to know about our internship program?"

        elif intent == 'thanks':
            return "You're very welcome! I'm glad I could help. Feel free to ask if you have any other questions about our internship program. Good luck with your application!"

        match_idx, similarity = self.find_best_match(user_query)

        if match_idx is not None and similarity > 0.15:
            faq_item = self.faq_data[match_idx]
            return f"**{faq_item['category']}**\n\n{faq_item['answer']}\n\n*Is there anything else you'd like to know about internships?*"

        keyword_match_idx, keyword_score = self.keyword_match(user_query)

        if keyword_match_idx is not None and keyword_score > 0:
            faq_item = self.faq_data[keyword_match_idx]
            return f"**{faq_item['category']}**\n\n{faq_item['answer']}\n\n*Would you like more information about this topic?*"

        return self.generate_fallback_response(intent, user_query)

    def generate_fallback_response(self, intent, user_query):
        fallback_responses = {
            'application': "For application-related questions, I can help with the application process, required documents, deadlines, and submission guidelines. Could you be more specific about what aspect of the application process you'd like to know about?",

            'requirements': "Regarding requirements, I can provide information about eligibility criteria, GPA requirements, academic qualifications, and other prerequisites. What specific requirements would you like to know about?",

            'compensation': "About compensation, our internships are paid positions with competitive rates that vary by role and location. We also offer additional benefits like mentorship, training, and potential full-time opportunities. Would you like more details about compensation structure?",

            'timeline': "For timeline information, application deadlines vary by program. Summer internships typically have fall deadlines, while other programs may have rolling admissions. When are you planning to intern?",

            'remote': "We offer flexible work arrangements including remote, hybrid, and on-site options depending on the role. Remote interns receive full support and resources. Are you interested in remote internship opportunities?",

            'projects': "Interns work on meaningful projects including real client work, product development, research, and process improvements. Projects are designed to provide hands-on learning. What type of projects interest you most?",

            'support': "We provide comprehensive support including mentorship, training, regular feedback, and professional development resources. What kind of support are you most interested in learning about?"
        }

        if intent in fallback_responses:
            return fallback_responses[intent]

        return """I'd be happy to help you with your internship questions! I can provide detailed information about:

• **Application Process** - How to apply, required documents, deadlines, submission tips
• **Requirements** - Eligibility criteria, GPA requirements, qualifications needed
• **Program Details** - Duration, schedule, compensation, and benefits
• **Remote Work** - Virtual internship options and flexible arrangements
• **Training & Development** - Learning opportunities, skill building, mentorship
• **Projects** - Types of meaningful work you'll be doing
• **Support** - Mentorship, guidance, and resources available
• **Career Opportunities** - Path to full-time positions and professional growth
• **Timeline** - Application deadlines and program schedules
• **Locations** - Office locations, housing assistance, relocation support

Please feel free to ask about any of these topics or be more specific about what you'd like to know!"""

print("Initializing Offline Internship Support Chatbot...")
chatbot = OfflineInternshipChatbot()

def test_chatbot():
    test_queries = [
        "Hello!",
        "How do I apply for an internship?",
        "What documents do I need?",
        "Are internships paid?",
        "Can I work remotely?",
        "What training do you provide?",
        "How long do internships last?",
        "What projects will I work on?",
        "When should I apply?",
        "Thank you for your help!"
    ]

    print("Testing chatbot with sample queries:")
    print("="*50)

    for query in test_queries:
        response = chatbot.generate_response(query)
        print(f"Q: {query}")
        print(f"A: {response[:150]}...")
        print("-" * 30)

def chat_interface(message, history):
    response = chatbot.generate_response(message)
    history.append((message, response))
    return history, ""

with gr.Blocks(title="Internship Support Chatbot", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🤖 Internship Support Chatbot")
    gr.Markdown("*Powered by offline AI - No downloads, no authentication errors!*")
    gr.Markdown("Ask me anything about our internship program! I can help with applications, requirements, training, compensation, and more.")

    chatbot_interface = gr.Chatbot(
        value=[("System", "Hello! I'm your internship support assistant. I can help you with questions about applications, requirements, training, projects, and more. What would you like to know?")],
        label="Chat with Internship Support Bot",
        height=500,
        show_label=True
    )

    msg = gr.Textbox(
        label="Your Message",
        placeholder="Type your internship question here...",
        lines=2
    )

    msg.submit(chat_interface, [msg, chatbot_interface], [chatbot_interface, msg])

    gr.Examples(
        examples=[
            "How do I apply for an internship?",
            "What documents do I need to submit?",
            "Are internships paid?",
            "Can I work remotely?",
            "What kind of training will I receive?",
            "How long do internships last?",
            "What projects will I work on?",
            "When should I apply for summer internships?",
            "What support is available for interns?",
            "Can internships lead to full-time jobs?"
        ],
        inputs=msg,
        label="Example Questions - Click to Try!"
    )

    gr.Markdown("---")
    gr.Markdown("### 📋 Topics I can help with:")
    gr.Markdown("""
    - **Application Process** - Step-by-step guidance
    - **Requirements** - Eligibility and qualifications
    - **Documentation** - Required papers and formats
    - **Timeline** - Deadlines and application periods
    - **Compensation** - Pay, benefits, and stipends
    - **Work Arrangements** - Remote, hybrid, on-site options
    - **Training** - Skills development and mentorship
    - **Projects** - Real work experience and responsibilities
    - **Support** - Guidance, resources, and assistance
    - **Career Growth** - Full-time opportunities and advancement
    """)

    gr.Markdown("*This chatbot uses offline text processing - fast, reliable, and no authentication required!*")

if __name__ == "__main__":
    print("="*60)
    print("🚀 OFFLINE INTERNSHIP SUPPORT CHATBOT")
    print("="*60)
    print("✅ No model downloads required")
    print("✅ No API keys needed")
    print("✅ No authentication errors")
    print("✅ Fast offline processing")
    print("="*60)

    test_chatbot()

    print("🎉 Launching Gradio interface...")
    demo.launch(share=True, debug=True)

Loading offline chatbot (no model downloads required)...
Initializing Offline Internship Support Chatbot...
Initializing chatbot...
Setting up text similarity models...
Processed 14 questions for similarity matching
Chatbot initialized successfully!
🚀 OFFLINE INTERNSHIP SUPPORT CHATBOT
✅ No model downloads required
✅ No API keys needed
✅ No authentication errors
✅ Fast offline processing
Testing chatbot with sample queries:
Q: Hello!
A: Hello! I'm your internship support assistant. I can help you with questions about applications, requirements, training, projects, compensation, and mu...
------------------------------
Q: How do I apply for an internship?
A: Hello! I'm your internship support assistant. I can help you with questions about applications, requirements, training, projects, compensation, and mu...
------------------------------
Q: What documents do I need?
A: **Application Process**

Required documents include: Resume/CV (PDF format, max 2 pages), Cover letter (tailored to t